In [15]:
import numpy as np
import pandas as pd
from pyhive import presto

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


conn = presto.connect(
host='presto.processing.yoda.run', 
## presto.processing.yoda.run
## bi-presto.serving.data.production.internal
port=80,
username='manoj.ravirajan@rapido.bike'
)

In [190]:
city_name = 'Lucknow'

In [219]:
customer_base_query = f"""

select 
    run_date execution_date,
    taxi_lifetime_last_ride_city city,
    customer_id,
    'PHH' segment

from
    hive.datasets.iallocator_customer_segments
WHERE run_date = '2022-09-04'
and taxi_lifetime_last_ride_city = '{city_name}'
and taxi_lifetime_stage in ('COMMITTED','SUSTENANCE','HOOK','DETOX','CHURN_OTB','SOFT_CHURN')

"""

customer_base = pd.read_sql(customer_base_query, conn)

customer_base.head(10)

,execution_date,city,customer_id,segment
0,2022-09-04,Lucknow,62fe0d0c53ac22a9dba043d0,PHH
1,2022-09-04,Lucknow,62fe0df968187b2380bb96d9,PHH
2,2022-09-04,Lucknow,62fe177353ac22593ba04ba5,PHH
3,2022-09-04,Lucknow,62fe297053ac22fd41a059b3,PHH
4,2022-09-04,Lucknow,62fe71ef53ac2206d4a087b6,PHH
5,2022-09-04,Lucknow,62fee3fc04cb1b0bf20a64e1,PHH
6,2022-09-04,Lucknow,62ff0c78856192e77a96e3f1,PHH
7,2022-09-04,Lucknow,62ff3c4204cb1b00eb0aaed6,PHH
8,2022-09-04,Lucknow,62ff9df98561922fec9751e7,PHH
9,2022-09-04,Lucknow,62ffb2a032e8240de0e8c065,PHH


In [192]:
customer_base_query = f"""

select 
    distinct customer_id customer_d1,
    'PHH' segment

from
    hive.datasets.iallocator_customer_segments
WHERE run_date between '2022-09-05' and '2022-09-11' 
and taxi_lifetime_last_ride_city = '{city_name}'
and taxi_lifetime_stage in ('COMMITTED','SUSTENANCE','HOOK','DETOX','CHURN_OTB','SOFT_CHURN')

"""

next_day = pd.read_sql(customer_base_query, conn)

next_day.head(10)

,customer_d1,segment
0,629373932b800fedb6f87fe7,PHH
1,6173ce284c6ba177dedcab7e,PHH
2,617603a24c6ba17e74de2733,PHH
3,61c6a0ef85de1274f202751a,PHH
4,61bc6b97907accfa2cc44f3c,PHH
5,61c93785872e3715c6fc2806,PHH
6,61d5a32b731a6d1306062a0f,PHH
7,627b6619d2e8d210a971f15c,PHH
8,628bb41e8f9b4367ad629218,PHH
9,6295895574c071262d504b4f,PHH


In [220]:
print(customer_base.shape,next_day.shape)

(56169, 4) (66870, 2)


In [221]:
customer_combined = customer_base.merge(next_day, left_on = 'customer_id',right_on = 'customer_d1',how='outer')

In [222]:
customer_combined.head()

,execution_date,city,customer_id,segment_x,customer_d1,segment_y
0,2022-09-04,Lucknow,62fe0d0c53ac22a9dba043d0,PHH,62fe0d0c53ac22a9dba043d0,PHH
1,2022-09-04,Lucknow,62fe0df968187b2380bb96d9,PHH,62fe0df968187b2380bb96d9,PHH
2,2022-09-04,Lucknow,62fe177353ac22593ba04ba5,PHH,62fe177353ac22593ba04ba5,PHH
3,2022-09-04,Lucknow,62fe297053ac22fd41a059b3,PHH,62fe297053ac22fd41a059b3,PHH
4,2022-09-04,Lucknow,62fe71ef53ac2206d4a087b6,PHH,62fe71ef53ac2206d4a087b6,PHH


In [223]:
customer_combined.customer_d1.nunique()

66870

In [224]:
customer_combined.customer_d1.count()

66870

In [225]:
customer_combined.isna().sum()

execution_date    11904
city              11904
customer_id       11904
segment_x         11904
customer_d1        1203
segment_y          1203
dtype: int64

In [226]:
customer_combined.fillna('missing',inplace=True)

In [227]:
customer_combined.groupby(['segment_x','segment_y']).agg({'customer_id':'count','customer_d1':'count'}).reset_index()

,segment_x,segment_y,customer_id,customer_d1
0,PHH,PHH,54966,54966
1,PHH,missing,1203,1203
2,missing,PHH,11904,11904
